# 百度网盘AI大赛-文档图片去遮挡比赛第3名方案


# 一、赛题分析
此次大赛主题结合日常生活常见情景展开，人们在使用手机等移动设备扫描证件或者扫描文档、拍摄展示资料的场景中，经常会拍摄到一些手指或者人头等其他因素，对扫描成品的美观和易用性产生了影响。期望同学们通过计算机技术对给定文档图像进行处理，帮助人们去除文档图像中的手指、人头等因素，还原真实的文档资料，提升使用效率。



# 二、 数据分析
- 本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共14400组样本，A榜测试集共320个样本，B榜测试集共640个样本,抽取一部分数据如图：
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a.jpg)
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a_0_000.jpg)
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a_0_000.png)

- hand,head 为带有手指、人头遮挡数据及遮挡部位分割图，gt 为非遮挡图片（仅有训练集数据提供gt ，A榜测试集、B榜测试集数据均不提供gt);
- annotation.txt 为图片对应关系，每一行为一组样本，用空格分隔，分别为非遮挡图片、遮挡图片、分割图片;

# 二、评价标准
评价指标为 PSNR 和 MSSSIM；

用于评价的机器环境仅提供两种框架模型运行环境：paddlepaddle 和 onnxruntime，其他框架模型可转换为
上述两种框架的模型；

机器配置：V100，显存16G，内存10G；

单张图片耗时>2s，决赛中的性能分数记0分。

由评价标准可知，不能使用大模型。

# 三、模型设计

针对图像去遮挡这个任务，我们查阅了相关资料，认为该任务是一个先擦出后重建优化的过程，所以我们选择了目前常用的EraseNet作为我们此次的baseline。整体思路分为预测目标所在位置，裁剪目标所在区域，初步去遮挡，去遮挡优化，生成mask，替换mask所在区域。
- 预测目标所在位置：
测试图片直接缩放成640x640进行预测并获得boundingbox。
预测效果如图所示：
- 裁剪目标所在区域
这里分为以下几种情况：
1）预测的boundingbox的宽高都小于1280
   目标区域向四周扩充为1280x1280。
2）预测的boundingbox的宽高有一个大于或者全都大于1280
    大于1280的边长向外扩充为64的倍数即可（避免出现维度不匹配的情况出现）
3）预测出多个boundingbox
    将多个boundingbox按面积由大到小排序，面积最大的boundingbox最先扩充，如果后面的boundingbox包含在这个扩充的boundingbox里面，则不进行处理，如果不包含其中，将其扩充为同样的尺寸，保证输入到模型中的宽高一致。
- 初步去遮挡
    改进后的EraseNet去除了预测mask的分支（训练中发现预测mask和去遮挡有冲突），整体可以看成unet结构。
- 去遮挡优化网络
    去遮挡优化网络采用U型结构的自监督学习的idr网络，通过再次的编解码进行去遮挡的二次优化，得到最终的去遮挡图。
- 生成mask
    利用yolox预测的boundbox(不扩充)在优化后的去遮挡图上裁剪出目标区域，将裁下的图片与输入图片目标所在区域做差，差值的绝对值大于等于2的地方作为mask。
- 替换mask所在区域
    将mask在原图中的位置的像素替换为mask在去遮挡图位置的像素。

![](![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/pipeline.png)

从网络结构图上可以直观的看出改进后EraseNet变成了单分支网络，这是因为原版EraseNet的预测mask分支和第一阶段的Decoder存在冲突，所以我们去掉了预测mask分支，考虑到实效性，我们没有额外训练一个分割模型，而是选择检测模型(yolox)来获得mask。在损失函数上，原版的ErastNet使用了感知损失以及GAN损失，这个损失函数是为了生成更加逼真的背景，但是本赛题任务的背景都是纯色，所以这两个损失是不需要的。此外，EraseNet在多个尺度上使用了l1损失，我们只在第一阶段和第二阶段的最后一个尺度上使用了l1损失。此外，根据经验，我们将EraseNet的重建网络Refinement替换为了idr网络并在底层叠加了non-local结构。



# 四、数据处理与增强

### 数据划分
1）yolox:
- 官方给的数据为14400张图片，所有图片直接缩放为640x640，其中12000张用于训练，2400张用于验证。
利用数据集自带的mask生成yolo格式标签,选用yolox作为检测模型。
2）EraseNet:
- 官方给的数据为14400张图像，我们裁剪出包含遮挡物的图片为10800张（全部为正样本，尺寸为1024x1024），其中9000张用作训练，1800张用于验证。

### 数据增广
- 增强使用横向左右翻转和小角度旋转

# 五、训练细节
- 训练配置
总迭代数：450000 iteration
我们采用batch size为4和patch size为1024来进行训练450000次迭代。
我们采用了余弦退火的学习率策略来优化网络，学习率我们采用1e-4，优化器为Adam。
- 损失函数为L1Loss

# 六、测试细节
- 原图缩放为640x640输入到yolox网络中获得boundingbox，除以缩放比例获得在原图上的boundingbox，将boundingbox所在区域扩充得到去遮挡网络的输入。
- 裁剪的图片输入到去遮挡网络中获得输出。
- 输出与输入做差获得mask.
- 将原图mask所在区域的像素值替换为输出mask所在区域的像素值。


# 七、代码结构
### erasenet_code:
- dataloader: 定义数据增强函数和数据集
- models: 定义网络模型
- loss:定义损失函数
- image_utils: 定义评价指标函数和其它函数
- checkpoint: 模型训练输出文件夹
- result:模型测试输出文件夹
- test.py: 预测脚本
- train.py: 训练脚本
### yolox_code:
- tools:训练测试脚本
- weights:预训练权重
- yolox:定义模型文件
- YOLOX_outputs:模型输出
- assets:测试图片存放位置
### test_code:
- predict.py: 预测脚本
- DataProcess:前处理和后处理
- model.onnx: 去遮挡网络最佳权重
- my_yolox_s.onnx:检测网络最佳权重

# 八、上分策略

![](https://ai-studio-static-online.cdn.bcebos.com/2bffa4513a1f498da0945379160fa87e420dd753765d4604bc5071e76e07de34)


上分策略主要集中在数据清洗、推理速度优化和损失函数优化

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 代码启动过程
## erasenet训练过程

In [1]:
### 解压数据
!cd datasets/ && unzip -q train*.zip

^C


In [3]:
##创建训练数据目录
! mkdir -p datasets/IMG
! mkdir -p datasets/TAR

In [5]:
##裁剪训练数据集
import os
import cv2
import numpy as np
constprefix="train_data_"
rgb_dir='./datasets'
IMG='./datasets/IMG'
TAR='./datasets/TAR'
#####裁剪尺寸为1024x1024
BASE=1024
for i in range(1,6):
    image_path=os.path.join(rgb_dir,constprefix + str(i))
    annotation_path=os.path.join(image_path,"annotation.txt")
    tar_filenames=[]
    inp_filenames=[]
    mask_filenames=[]
    with open (annotation_path,"r") as f:
            data=f.read().splitlines()
            for i in range (len(data)):
                cur=data[i].split(' ')
                tar_filenames.append(os.path.join(image_path,cur[0]))
                inp_filenames.append(os.path.join(image_path,cur[1]))
                mask_filenames.append(os.path.join(image_path,cur[2]))
    
    for i in range(len(mask_filenames)):
    
        mask_path=mask_filenames[i]
        inp_path=inp_filenames[i]
        tar_path=tar_filenames[i]
        filename=os.path.split(mask_path)[-1]
        mask_img=cv2.imread(mask_path,0)
        inp_img=cv2.imread(inp_path)
        tar_img=cv2.imread(tar_path)
        H,W=mask_img.shape
        mask_img[np.where(mask_img>40)]=255
        mask_img[np.where(mask_img<40)]=0
        contours, hierarchy = cv2.findContours(mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x, y, w, h = cv2.boundingRect(contours[0])
        x1=x;x2=x+w;y1=y;y2=y+h;
        w_add_half=BASE//2
        h_add_half=BASE//2
        x_mid=(x1+x2)//2
        y_mid=(y1+y2)//2
        if(x_mid-w_add_half<=0):
            x1=0
            x2=BASE
        elif(x_mid+w_add_half>W):
            x2=W
            x1=W-BASE
        else:
            x1=x_mid-w_add_half
            x2=x_mid+w_add_half
        if(y_mid-h_add_half<=0):
            y1=0
            y2=BASE
            
        elif(y_mid+h_add_half>H):
            y2=H
            y1=H-BASE
            
        else:
            y1=y_mid-h_add_half
            y2=y_mid+h_add_half
        inp_filename=os.path.split(inp_path)[-1]
        tar_filename=inp_filename.split('.jpg')[0]+'_tar.jpg'
        new_inp_path=os.path.join(IMG,inp_filename)
        new_tar_path=os.path.join(TAR,tar_filename)
        new_inp_img=inp_img[y1:y2,x1:x2,:]
        new_tar_img=tar_img[y1:y2,x1:x2,:]
        print(new_tar_path)
        cv2.imwrite(new_inp_path,new_inp_img)
        cv2.imwrite(new_tar_path,new_tar_img)

KeyboardInterrupt: 

In [1]:
## 安装训练所需的python包
! pip install torch==1.8.1
! pip install torchvision==0.9.1
! pip install natsort  
! pip install opencv-python
! pip install scikit-image
! pip install warmup_scheduler
! pip install pillow

^C
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2998 sha256=e51155ddd5e11b5b054fc3308df854a41a5ec4e7f4f81ba9f62f2fe38d60363b
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\af\09\35\3ef9059c328587fc37578f2e6b4039b3fbe6485a99ae9fc41e
Successfully built warmup-scheduler
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [6]:
## 利用L1Loss训练
! cd erasenet_code/ && python train.py 

Loading weights into state dict...


Traceback (most recent call last):
  File "train.py", line 24, in <module>
    yolo = YOLO()
  File "c:\Users\Administrator\Desktop\BaiduDiskAI_DocRemoveCover_top3\erasenet_code\models\yolo.py", line 48, in __init__
    self.generate()
  File "c:\Users\Administrator\Desktop\BaiduDiskAI_DocRemoveCover_top3\erasenet_code\models\yolo.py", line 77, in generate
    self.net = self.net.cuda()
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 491, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 387, in _apply
    module._apply(fn)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 387, in _apply
    module._apply(fn)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 409, in _apply
    param_applied = fn(param)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 491,

## yolox训练过程

In [1]:
##创建训练数据目录
! mkdir -p datasets/VOCdevkit/VOC2007/Annotations
! mkdir -p datasets/VOCdevkit/VOC2007/ImageSets
! mkdir -p datasets/VOCdevkit/VOC2007/JPEGImages
! mkdir -p datasets/VOCdevkit/VOC2007/Mask

�����﷨����ȷ��
�����﷨����ȷ��
�����﷨����ȷ��
�����﷨����ȷ��


In [5]:
##划分数据集
import os
import cv2
import numpy as np
constprefix="train_data_"
rgb_dir='./datasets'
inp='./datasets/VOCdevkit/VOC2007/JPEGImages'
mask='./datasets/VOCdevkit/VOC2007/Mask'

for i in range(1,6):
    image_path=os.path.join(rgb_dir,constprefix + str(i))
    annotation_path=os.path.join(image_path,"annotation.txt")
    inp_filenames=[]
    mask_filenames=[]
    with open (annotation_path,"r") as f:
            data=f.read().splitlines()
            for i in range (len(data)):
                cur=data[i].split(' ')
                inp_filenames.append(os.path.join(image_path,cur[1]))
                mask_filenames.append(os.path.join(image_path,cur[2]))
    
    for j in range(len(mask_filenames)):
        mask_path=mask_filenames[j]
        inp_path=inp_filenames[j]
        mask_img=cv2.imread(mask_path,0)
        inp_img=cv2.imread(inp_path)
        mask_img[np.where(mask_img>40)]=255
        mask_img[np.where(mask_img<40)]=0
        inp_filename=os.path.split(inp_path)[-1]
        mask_filename=os.path.split(mask_path)[-1]
        new_inp_path=os.path.join(inp,inp_filename)
        new_mask_path=os.path.join(mask,mask_filename)
        cv2.imwrite(new_inp_path,inp_img)
        cv2.imwrite(new_mask_path,mask_img)


./datasets/VOCdevkit/VOC2007/JPEGImages/0_IMG_20220705_103123_a_0_000.jpg
./datasets/VOCdevkit/VOC2007/JPEGImages/0_IMG_20220705_103123_a_0_001.jpg
./datasets/VOCdevkit/VOC2007/JPEGImages/0_IMG_20220705_103123_a_0_002.jpg
./datasets/VOCdevkit/VOC2007/JPEGImages/0_IMG_20220705_103123_a_0_003.jpg
./datasets/VOCdevkit/VOC2007/JPEGImages/0_IMG_20220705_103123_a_0_004.jpg
./datasets/VOCdevkit/VOC2007/JPEGImages/0_IMG_20220705_103123_a_0_005.jpg


KeyboardInterrupt: 

In [7]:
##mask2voc
import os
import numpy as np
from itertools import groupby
from skimage import morphology,measure
from PIL import Image
from scipy import misc
import xml.etree.ElementTree as ET
import os
from pdb import set_trace as stx
import copy

def createXMLlabel(savedir,objectnum, contours, classname, foldername='0',filename='0', path='0', database='road', width='400', height='600',depth='3', segmented='0', pose="Unspecified", truncated='0', difficult='0'):
    # 创建根节点
    root = ET.Element("annotation")
 
    # 创建子节点
    folder_node = ET.Element("folder")
    folder_node.text = foldername
    # 将子节点数据添加到根节点
    root.append(folder_node)
 
    file_node = ET.Element("filename")
    file_node.text = filename
    root.append(file_node)
    path_node = ET.Element("path")
    path_node.text = path
    root.append(path_node)
 
    source_node = ET.Element("source")
    # 也可以使用SubElement直接添加子节点
    db_node = ET.SubElement(source_node, "database")
    db_node.text = database
    root.append(source_node)
 
    size_node = ET.Element("size")
    width_node = ET.SubElement(size_node, "width")
    height_node = ET.SubElement(size_node, "height")
    depth_node = ET.SubElement(size_node, "depth")
    width_node.text = width
    height_node.text = height
    depth_node.text = depth
    root.append(size_node)
 
    seg_node = ET.Element("segmented")
    seg_node.text = segmented
    root.append(seg_node)
 
    for i in range(objectnum):
        x, y, w, h = cv2.boundingRect(contours[i]) 
        newEle = ET.Element("object")
        name = ET.Element("name")
        name.text = classname
        newEle.append(name)
        pose_node = ET.Element("pose")
        pose_node.text = pose
        newEle.append(pose_node)
        trunc = ET.Element("truncated")
        trunc.text = truncated
        newEle.append(trunc)
        dif = ET.Element("difficult")
        dif.text = difficult
        newEle.append(dif)
        boundingbox = ET.Element("bndbox")
        xmin = ET.SubElement(boundingbox, "xmin")
        ymin = ET.SubElement(boundingbox, "ymin")
        xmax = ET.SubElement(boundingbox, "xmax")
        ymax = ET.SubElement(boundingbox, "ymax")
        xmin.text = str(x)
        ymin.text = str(y)
        xmax.text = str(x + np.round(w).astype(int))
        ymax.text = str(y + np.round(h).astype(int))
        newEle.append(boundingbox)
        root.append(newEle)
 
    ImageID = filename.split('.')[0]
    # 创建elementtree对象，写入文件
    tree = ET.ElementTree(root)
    tree.write(savedir + '/'+ ImageID + ".xml")


if __name__=="__main__":
    import cv2
    imagedir = './datasets/VOCdevkit/VOC2007/Mask'
    saveXMLdir = './datasets/VOCdevkit/VOC2007/Annotations'
    if os.path.exists(saveXMLdir) is False:
        os.mkdir(saveXMLdir)

    for fname in sorted(os.listdir(imagedir)):
        labelpath = os.path.join(imagedir, fname)
        image = cv2.imread(labelpath,0)
        # 得到label图上的boundingingbox和数量
   
        contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        objectnum=len(contours)
        # label图 命名格式为 ImgeID_classname.png
        # for i in range(objectnum):
        #     x, y, w, h = cv2.boundingRect(contours[i])
        #     cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        ImageID = os.path.split(labelpath)[-1].split('.png')[0]
        classname = 'hh'
        origin_image_name = ImageID +'.jpg'

        # 一些图片信息
        foldername = 'test_dataset'
        path  ='./datasets/VOCdevkit/VOC2007/JPEGImages/'+ origin_image_name
        database = 'Unknown'
        width = str(image.shape[0])
        height = str(image.shape[1])
        depth = str(1)

        createXMLlabel(saveXMLdir,objectnum, contours, classname, foldername=foldername,filename=origin_image_name, path=path,
                    database=database, width=width, height=height,depth=depth, segmented='0', pose="Unspecified",
                    truncated='0', difficult='0')

In [8]:
##voc2yolo
import os
import random
random.seed(0)
 
xmlfilepath = r'./datasets/VOCdevkit/VOC2007/Annotations'
saveBasePath = r"./datasets/VOCdevkit/VOC2007/ImageSets/Main"
 
trainval_percent = 0.8
train_percent = 1
 
temp_xml = os.listdir(xmlfilepath)
total_xml = []
for xml in temp_xml:
    if xml.endswith(".xml"):
        total_xml.append(xml)
 
num = len(total_xml)
list = range(num)
tv = int(num*trainval_percent)
tr = int(tv*train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)
 
print("train and val size", tv)
print("traub suze", tr)
ftrainval = open(os.path.join(saveBasePath, 'trainval.txt'), 'w')
ftest = open(os.path.join(saveBasePath, 'test.txt'), 'w')
ftrain = open(os.path.join(saveBasePath, 'train.txt'), 'w')
fval = open(os.path.join(saveBasePath, 'val.txt'), 'w')
 
for i in list:
    name = total_xml[i][:-4]+'\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)
 
ftrainval.close()
ftrain.close()
fval.close()

train and val size 4
traub suze 4


In [4]:
## 安装训练所需的python包
! pip install loguru
! pip install tensorboard
! pip install pycocotools
! pip install tabulate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build pycocotools


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' 'C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\Administrator\AppData\Local\Temp\tmpe5w_mwk4'
       cwd: C:\Users\Administrator\AppData\Local\Temp\pip-install-wo3m8gba\pycocotools_46172c95f4da41849319517e4ced8d23
  Complete output (16 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-38
  creating build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\coco.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\cocoeval.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\mask.py -> build\lib.win-amd64-cpython-38\pycocotools
  copying pycocotools\__init__.py -> build\lib.win-amd64-cpython-38\pycocotools
  running build_ext
  cythoning pycocotools/_mask.pyx to pycocotools\_mask.c
  C:\Users\Administrator\AppData\Local\Temp\p

In [8]:
## 训练yolox
! cd yolox_code/ && python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -d 0 -b 8 --fp16 -o -c weights/yolox_s.pth 

 

c:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:116: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
2022-10-13 20:06:43 | INFO     | yolox.core.trainer:130 - args: Namespace(batch_size=8, cache=False, ckpt='weights/yolox_s.pth', devices=0, dist_backend='nccl', dist_url=None, exp_file='exps/example/yolox_voc/yolox_voc_s.py', experiment_name='yolox_voc_s', fp16=True, logger='tensorboard', machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2022-10-13 20:06:43 | INFO     | yolox.core.trainer:131 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './

## 预测过程
按照官方给定的预测脚本运行方式相同，将最终的finetuning模型改名为model.pdparams，并且放在predict.py的同意目录下，使用命令：
```
python predict.py [src_image_dir] [results]
```

In [ ]:
## 测试脚本
! cd test_code/ && python predict.py {your_test_data_path} {save_path}

: 